In [1]:
from pynq import Overlay, allocate, PL
import struct
import numpy
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

PL.reset()
overlay = Overlay('cnn_fpga.bit')

In [2]:
print('IP blocks :', list(overlay.ip_dict.keys()))

IP blocks : ['MaxPooling2D_0', 'conv2D_3x3_0', 'ReLU_0', 'Linear_0', 'axi_dma_conv2D', 'axi_dma_relu', 'axi_dma_maxpool2D', 'axi_dma_linear', 'processing_system7_0']


In [3]:
def float_to_hex(f):
    return struct.unpack('I', struct.pack('f', f))[0]

def hex_to_float(f):
    return struct.unpack('f', struct.pack('I', f))[0]

def numpy_to_hex(f):
    #assert f.shape
    format_string = f'{f.shape[0]}f'
    packed_data = struct.pack(format_string, *f)
    return packed_data

def hex_to_numpy(f, length):
    format_string = f'{length}f'
    unpacked_data = struct.unpack(format_string, f)
    return np.array(unpacked_data)

data_in = np.array([10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0], dtype=np.float32)
data_hex = numpy_to_hex(data_in)
print(data_hex)
data_out = hex_to_numpy(data_hex, 9)
print(data_out)

b'\x00\x00 A\x00\x000A\x00\x00@A\x00\x00PA\x00\x00`A\x00\x00pA\x00\x00\x80A\x00\x00\x88A\x00\x00\x90A'
[10. 11. 12. 13. 14. 15. 16. 17. 18.]


In [12]:
# Load the .pt file
#with open("mnist_cnn.pt", "rb") as f:
#    data = pickle.load(f)

#load the pre-processed pickle
with open("mnist_cnn.pkl", "rb") as f:
    data = pickle.load(f)
    
# Print the loaded data structure
print(type(data))
print(data.keys())

if isinstance(data, dict):
    for key in data:
        print(f"{key}: {type(data[key])}")

# Example: Convert a specific weight tensor to a NumPy array
weight_key = "conv1.weight"  # Replace with the key you want to inspect

weight_tensor = data[weight_key]
print(len(weight_tensor))
print(len(weight_tensor[0]))
print(weight_tensor[0][0])
# Convert to a NumPy array
weight_array = np.array(weight_tensor[0][0])
print(weight_array)

# Example: Convert a specific weight tensor to a NumPy array
weight_key = "fc1.weight"  # Replace with the key you want to inspect

weight_tensor = data[weight_key]
print(len(weight_tensor))
print(len(weight_tensor[0]))
print(weight_tensor[0][0])
# Convert to a NumPy array
weight_array = np.array(weight_tensor[0][0])
print(weight_array)
    
#if hasattr(data[weight_key], 'numpy'):
#    weight_array = data[weight_key].numpy()  # Convert directly to NumPy array
#else:
#    print("The data format isn't directly convertible. Raw data might need more processing.")

<class 'dict'>
dict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])
conv1.weight: <class 'list'>
conv1.bias: <class 'list'>
conv2.weight: <class 'list'>
conv2.bias: <class 'list'>
fc1.weight: <class 'list'>
fc1.bias: <class 'list'>
fc2.weight: <class 'list'>
fc2.bias: <class 'list'>
32
1
[[0.1870943158864975, -0.23402291536331177, -0.2610444724559784], [0.14852683246135712, -0.34062135219573975, 0.17761141061782837], [-0.07393540441989899, 0.22221820056438446, 0.11669355630874634]]
[[ 0.18709432 -0.23402292 -0.26104447]
 [ 0.14852683 -0.34062135  0.17761141]
 [-0.0739354   0.2222182   0.11669356]]
128
12544
0.0047676535323262215
0.0047676535323262215


In [ ]:
class Module():
    def __init__(self, overlay):
        self.overlay = overlay
        self.weights = []
        self.bias = []
        self.output_buffer = None
        self.layer_ip = None
        self.ip_dict = None
        self.dma_send = None
        self.dma_recv = None
        
    def __call__(self, x):
        return self.forward(x)
    
    def forward(self, x):
        pass
    
    #def get_register_offset(self, ip, parameter):
    #    #print(self.overlay.ip_dict[ip]['registers'])
    #    return self.overlay.ip_dict[ip]['registers'][parameter]['address_offset']
        
    def read_param_float(self, param):
        address = self.ip_dict['registers'][param]['address_offset']
        data = self.layer_ip.read(address)
        return hex_to_float(data)
        
    def read_param_hex(self, param):
        address = self.ip_dict['registers'][param]['address_offset']
        data = self.layer_ip.read(address)
        return data

    def write_param_float(self, param, value):
        address = self.ip_dict['registers'][param]['address_offset']
        self.layer_ip.write(address, float_to_hex(value))
    
    def write_param_hex(self, param, value):
        address = self.ip_dict['registers'][param]['address_offset']
        self.layer_ip.write(address, value)
    
    def write_param_numpy(self, param, values):
        #address = self.ip_dict['registers'][param]['address_offset']
        #self.layer_ip.write(address, numpy_to_hex(values))
        
        address = self.ip_dict['registers'][param]['address_offset']
        for i in range(values.size):
            print("writting: ", values[i], " to: ", address+4*i)
            self.layer_ip.write(address+4*i, float_to_hex(values[i]))
        
    def read_param_numpy(self, param, length=1):
        address = self.ip_dict['registers'][param]['address_offset']
        data = []
        for i in range(length):
            data_ = self.layer_ip.read(offset=address + 4*i)
            data.append(hex_to_float(data_))
        #return hex_to_numpy(data)
        #return hex_to_float(data)
        return np.array(data, dtype=np.float32)
        
    def process_ip(self, in_buffer, out_buffer):
        self.layer_ip.write(0x0, 0x01)
        self.dma_send.transfer(in_buffer)
        self.dma_recv.transfer(out_buffer)
        print(type(self).__name__, " sending")
        self.dma_send.wait()
        print(type(self).__name__, " recieving")
        self.dma_recv.wait()
    
    
class Linear_(Module):
    def __init__(self, overlay, in_size, out_size):
        Module.__init__(self, overlay)
        
        self.in_size = in_size
        self.out_size = out_size
        self.output_buffer = []
        self.output_buffer.append(allocate(shape=(out_size,), dtype=np.float32))
    
        self.layer_ip = overlay.Linear_0
        self.ip_dict = overlay.ip_dict['Linear_0']

        self.dma_send = overlay.axi_dma_linear.sendchannel
        self.dma_recv = overlay.axi_dma_linear.recvchannel
            
    def forward(self, x):
        
        assert len(x) == 1 and x[0].shape[0] == self.in_size
        
        #print("weights: ", self.weights[0])
        #print("bias: ", self.bias[0])
                
        self.write_param_numpy('Memory_weights', self.weights.reshape(-1))
        self.write_param_numpy('Memory_bias', self.bias.reshape(-1))   
        self.write_param_hex('in_size', self.in_size)
        self.write_param_hex('out_size', self.out_size)
        
        self.process_ip(x[0], self.output_buffer[0])
                            
        return self.output_buffer
   

class Linear(Module):
    def __init__(self, overlay, in_size, out_size):
        Module.__init__(self, overlay)
        
        self.in_size = in_size
        self.out_size = out_size
        self.output_buffer = allocate(shape=(out_size,), dtype=np.float32)
    
        self.weight_buffer = []
        for i in range(out_size):
            buffer = allocate(shape=(in_size,), dtype=np.float32)
            self.weight_buffer.append(buffer)
    
        self.layer_ip = overlay.dot_0
        self.ip_dict = overlay.ip_dict['dot_0']

        self.dma_send_1 = overlay.axi_dma_dot_in1.sendchannel
        self.dma_send_2 = overlay.axi_dma_dot_in2.sendchannel
    
    def setWeight(self, weight):
        assert len(weight.shape) == 2
        assert weight.shape[0] == self.in_size
        assert weight.shape[1] == self.out_size
        
        for i in range(self.out_size):
            self.weight_buffer[i] = weight[:,i]
        
    def process_ip(self, in_buffer1, in_buffer2):
        self.layer_ip.write(0x0, 0x01)
        self.dma_send_1.transfer(in_buffer1)
        self.dma_send_2.transfer(in_buffer2)
        print(type(self).__name__, " sending")
        self.dma_send_1.wait()
        self.dma_send_2.wait()
        #print(type(self).__name__, " recieving")
        #self.dma_recv.wait()
        
    def forward(self, x):
        
        assert len(x.shape) == 1
        assert x.shape[0] == self.in_size
        
        #print("weights: ", self.weights[0])
        #print("bias: ", self.bias[0])
                
        #self.write_param_numpy('Memory_weights', self.weights[0])
        #self.write_param_numpy('Memory_bias', self.weights[0])   
        self.write_param_hex('in_size', self.in_size)
        self.write_param_hex('out_size', self.out_size)
        
        #self.process_ip(x, self.output_buffer)
        
        for i in range(self.out_size):         
            print("weights: ", self.weights[i])
            print("bias: ", self.bias[i])
            #self.write_param_numpy('Memory_weights', self.weights[out_channel])   
            #self.write_param_float('bias', self.bias[out_channel])   
            #print(self.read_param_numpy('Memory_weights', 9))
            self.process_ip(x, self.weight_buffer[i])

            self.output_buffer[i] = self.read_param_float("result") + self.bias[i]
                    
        return self.output_buffer
        

class Conv2d(Module):
    def __init__(self, overlay, in_height, in_width, in_channels, out_channels, kernel_size=3, stride=1):
        Module.__init__(self, overlay)
        
        assert kernel_size == 3
        assert stride == 1
        
        self.layer_ip = overlay.conv2D_3x3_0
        self.ip_dict = overlay.ip_dict['conv2D_3x3_0']
        self.dma_send = self.overlay.axi_dma_conv2D.sendchannel
        self.dma_recv = self.overlay.axi_dma_conv2D.recvchannel
        
        self.in_width = in_width
        self.in_height = in_height
        self.in_channels = in_channels
        
        self.out_width = in_width
        self.out_height = in_height
        self.out_channels = out_channels
        
        self.output_buffer = allocate(shape=(self.out_channels, self.out_height, self.out_width), dtype=np.float32)
        self.aux_in_buffer = allocate(shape=(self.in_height, self.in_width), dtype=np.float32)
        self.aux_out_buffer = allocate(shape=(self.out_height, self.out_width), dtype=np.float32)

    def forward(self, x):
        
        assert len(x.shape) == 3 
        assert x.shape[0] == self.in_channels
        assert x.shape[1] == self.in_height
        assert x.shape[2] == self.in_width
        
        self.write_param_hex('in_width', self.in_width)
        self.write_param_hex('in_height', self.in_height)
        
        for out_channel in range(self.out_channels):      
            for in_channel in range(self.in_channels):   
                 
                print("weights: ", self.weights[out_channel][in_channel])
                print("bias: ", self.bias[out_channel])
                self.write_param_numpy('Memory_weights', self.weights[out_channel][in_channel])   
                #self.write_param_float('bias', self.bias[out_channel])   
                #print(self.read_param_numpy('Memory_weights', 9))
                self.aux_in_buffer = x[in_channel, :, :]
                self.process_ip(self.aux_in_buffer, self.aux_out_buffer)
                
                if in_channel == 0:
                    self.output_buffer[out_channel] = self.aux_out_buffer
                else:
                    self.output_buffer[out_channel] += self.aux_out_buffer
            
            self.output_buffer[out_channel] += self.bias[out_channel]
            
        return self.output_buffer        


class ReLU(Module):
    def __init__(self, overlay, data_size):
        Module.__init__(self, overlay)
    
        #fpga specific
        self.layer_ip = overlay.ReLU_0
        self.ip_dict = overlay.ip_dict['ReLU_0']
        self.dma_send = overlay.axi_dma_relu.sendchannel
        self.dma_recv = overlay.axi_dma_relu.recvchannel
        
        self.data_size = data_size
        self.output_buffer = allocate(shape=(self.data_size,), dtype=np.float32)
        
    def forward(self, x):
        
        assert sum(x.shape) == self.data_size 
        
        self.write_param_hex('data_size', self.data_size)

        self.process_ip(x.reshape(-1), self.output_buffer)
        self.output_buffer = self.output_buffer.reshape(x.shape)
            
        return self.output_buffer


class MaxPooling2D(Module):
    def __init__(self, overlay, in_height, in_width, in_channels):
        Module.__init__(self, overlay)
    
        self.layer_ip = overlay.MaxPooling2D_0
        self.ip_dict = overlay.ip_dict['MaxPooling2D_0']
        self.dma_send = overlay.axi_dma_maxpool2D.sendchannel
        self.dma_recv = overlay.axi_dma_maxpool2D.recvchannel
        
        self.in_height = in_height
        self.in_width = in_width
        self.in_channels = in_channels
        
        self.out_height = int(in_height/2)
        self.out_width = int(in_width/2)
        self.out_channels = in_channels
                
        self.output_buffer = allocate(shape=(self.out_channels, self.out_height, self.out_width), dtype=np.float32)
        
        self.aux_in_buffer = allocate(shape=(self.in_height, self.in_width), dtype=np.float32)
        self.aux_out_buffer = allocate(shape=(self.out_height, self.out_width), dtype=np.float32)
        
    def forward(self, x):
        
        assert len(x.shape) == 3
        assert x.shape[0] == self.in_channels
        assert x.shape[1] == self.in_height
        assert x.shape[2] == self.in_width
        
        self.write_param_hex('in_width', self.in_width)
        self.write_param_hex('in_height', self.in_height)

        for channel in range(self.in_channels):
            self.aux_in_buffer = x[channel, :, :]
            self.process_ip(self.aux_in_buffer, self.aux_out_buffer)
            self.output_buffer[channel, :, :] = self.aux_out_buffer
        
        return self.output_buffer

    
class Net(Module):
    def __init__(self, overlay):
        #super(Net, self).__init__()
        Module.__init__(self, overlay)
        self.conv1 = Conv2d(overlay, 28, 28, 1, 32, 3, 1)
        self.relu1 = ReLU(overlay, 28, 28, 32)
        self.conv2 = Conv2d(overlay, 28, 28, 32, 64, 3, 1)
        self.relu2 = ReLU(overlay, 28, 28, 64)
        #self.max_pool2d = MaxPooling2D(overlay, 64, 64, 64)
                
        #self.dropout1 = Dropout(0.25)
        #self.dropout2 = Dropout(0.5)
        self.fc1 = Linear(overlay, 28*28*32, 128)
        self.relu3 = ReLU(overlay, 128, 1, 1)
        self.fc2 = Linear(overlay, 128, 10)
        
        self.flatten_buffer = allocate(shape=(28*28*32,), dtype=np.float32)
    
    def load_state_dict(self, pickle_file):
        #load the pre-processed pickle
        with open(pickle_file, "rb") as f:
            data = pickle.load(f)
    
        conv1_weight = data["conv1.weight"]
        conv1_bias = data["conv1.bias"]
        
        assert len(conv1_weight) == self.conv1.out_channels 
        assert len(conv1_bias) == self.conv1.out_channels
        assert len(conv1_weight[0]) == self.conv1.in_channels
        #assert len(conv1_weight[0][0]) == 9
        #assert len(conv1_bias[0]) == 1
        
        self.conv1.weights = []
        self.conv1.bias = []
        for i in range(self.conv1.out_channels):
            self.conv1.weights.append([])
            for j in range(self.conv1.in_channels):
                self.conv1.weights[i].append(np.array(conv1_weight[i][j], dtype=np.float32).reshape(-1))
                self.conv1.bias.append(conv1_bias[i])

        conv2_weight = data["conv2.weight"]
        conv2_bias = data["conv2.bias"]
        
        assert len(conv2_weight) == self.conv2.out_channels 
        assert len(conv2_bias) == self.conv2.out_channels
        assert len(conv2_weight[0]) == self.conv2.in_channels
        #assert len(conv2_weight[0][0]) == 9
        #assert len(conv2_bias[0]) == 1
        
        self.conv2.weights = []
        self.conv2.bias = []
        for i in range(self.conv2.out_channels):
            self.conv2.weights.append([])
            for j in range(self.conv2.in_channels):
                self.conv2.weights[i].append(np.array(conv2_weight[i][j], dtype=np.float32).reshape(-1))
                self.conv2.bias.append(conv2_bias[i])
          
        fc1_weight = data["fc1.weight"]
        fc1_bias = data["fc1.bias"]
        
        assert len(fc1_weight) == self.fc1.out_size 
        assert len(fc1_bias) == self.fc1.out_size
        
        self.fc1.weights = []
        self.fc1.bias = []
        
        self.fc1.weights.append(np.array(fc1_weight, dtype=np.float32).reshape(-1))
        self.fc1.bias.append(np.array(fc1_bias, dtype=np.float32).reshape(-1))
    
        fc2_weight = data["fc2.weight"]
        fc2_bias = data["fc2.bias"]
        
        assert len(fc2_weight) == self.fc2.out_size 
        assert len(fc2_bias) == self.fc2.out_size
        
        self.fc2.weights = []
        self.fc2.bias = []

        self.fc2.weights.append(np.array(fc2_weight, dtype=np.float32).reshape(-1))
        self.fc2.bias.append(np.array(fc2_bias, dtype=np.float32).reshape(-1))
            
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        #x = self.conv2(x)
        #x = self.relu2(x)
        #x = self.max_pool2d(x)
        #x = self.dropout1(x)
        #x = torch.flatten(x, 1)
        for i in range(len(x)):
            self.flatten_buffer[i*28*28:(i+1)*28*28] = x[i]
        x = self.fc1([self.flatten_buffer])
        x = self.relu3(x)
        #x = self.dropout2(x)
        x = self.fc2(x)
        output = x #self.log_softmax(x, dim=1)
        return output

In [6]:
model = Net(overlay)
model.load_state_dict("mnist_cnn.pkl")

input_buffer = [allocate(shape=(28*28,), dtype=np.float32)]
image = model(input_buffer)

weights:  [ 0.18709432 -0.23402292 -0.26104447  0.14852683 -0.34062135  0.17761141
 -0.0739354   0.2222182   0.11669356]
bias:  -0.04103250801563263
writting:  1044354471  to:  64
writting:  3194987444  to:  68
writting:  3196430239  to:  72
writting:  1041766251  to:  76
writting:  3199100396  to:  80
writting:  1043718084  to:  84
writting:  3180817266  to:  88
writting:  1046711595  to:  92
writting:  1039072520  to:  96
Conv2d  sending
Conv2d  recieving
weights:  [-0.02203571  0.12048615  0.0602811   0.12666677 -0.14679286 -0.08213117
 -0.11653146  0.01846959 -0.03560807]
bias:  0.04075676202774048
writting:  3165946940  to:  64
writting:  1039581553  to:  68
writting:  1031203153  to:  72
writting:  1040299247  to:  76
writting:  3189133534  to:  80
writting:  3181917283  to:  84
writting:  3186534412  to:  88
writting:  1016548746  to:  92
writting:  3172063684  to:  96
Conv2d  sending
Conv2d  recieving
weights:  [ 0.376787    0.14397247 -0.15839463 -0.13235126 -0.16254672 -0.260

weights:  [-0.26191473  0.10403506 -0.08529423 -0.27254984 -0.02149849  0.2851964
 -0.28822422  0.17086178  0.27607727]
bias:  0.22719265520572662
writting:  3196459440  to:  64
writting:  1037373526  to:  68
writting:  3182341822  to:  72
writting:  3196816295  to:  76
writting:  3165658522  to:  80
writting:  1049756995  to:  84
writting:  3197342240  to:  88
writting:  1043265124  to:  92
writting:  1049451008  to:  96
Conv2d  sending
Conv2d  recieving
weights:  [ 0.31943735 -0.24542096  0.03050291 -0.23177539  0.2520463  -0.21037419
  0.2165868   0.01524869 -0.07565971]
bias:  -0.0852276086807251
writting:  1050905931  to:  64
writting:  3195752354  to:  68
writting:  1023009087  to:  72
writting:  3194836615  to:  76
writting:  1048644662  to:  80
writting:  3193400405  to:  84
writting:  1046333678  to:  88
writting:  1014617503  to:  92
writting:  3181048698  to:  96
Conv2d  sending
Conv2d  recieving
weights:  [-0.10009602 -0.0213446   0.04291826 -0.03477974  0.00822338 -0.39314

writting:  997867492  to:  3664
writting:  3155134099  to:  3668
writting:  3127911104  to:  3672
writting:  3146056371  to:  3676
writting:  1003266663  to:  3680
writting:  992970540  to:  3684
writting:  3155223698  to:  3688
writting:  3156026368  to:  3692
writting:  3145131045  to:  3696
writting:  3142420864  to:  3700
writting:  953279767  to:  3704
writting:  3125029679  to:  3708
writting:  978304454  to:  3712
writting:  996798426  to:  3716
writting:  999745752  to:  3720
writting:  990995428  to:  3724
writting:  1002856929  to:  3728
writting:  3156909673  to:  3732
writting:  991748679  to:  3736
writting:  996858804  to:  3740
writting:  3134490571  to:  3744
writting:  997567698  to:  3748
writting:  997596346  to:  3752
writting:  991287986  to:  3756
writting:  3146779684  to:  3760
writting:  972768537  to:  3764
writting:  3147002909  to:  3768
writting:  3154896452  to:  3772
writting:  3151416405  to:  3776
writting:  3109276748  to:  3780
writting:  3132574124  

writting:  3146244994  to:  7172
writting:  3143841272  to:  7176
writting:  993559195  to:  7180
writting:  3130102438  to:  7184
writting:  1006143967  to:  7188
writting:  993768252  to:  7192
writting:  1007222943  to:  7196
writting:  1004879149  to:  7200
writting:  3123371743  to:  7204
writting:  3151816117  to:  7208
writting:  1006645914  to:  7212
writting:  3146347838  to:  7216
writting:  3147277709  to:  7220
writting:  1005183001  to:  7224
writting:  992231300  to:  7228
writting:  3133397632  to:  7232
writting:  1000966293  to:  7236
writting:  997897678  to:  7240
writting:  3150866972  to:  7244
writting:  1005910616  to:  7248
writting:  3153161005  to:  7252
writting:  3149819278  to:  7256
writting:  985237849  to:  7260
writting:  3148061279  to:  7264
writting:  3124730274  to:  7268
writting:  1003703793  to:  7272
writting:  1004158138  to:  7276
writting:  3152623202  to:  7280
writting:  1001584039  to:  7284
writting:  1002181985  to:  7288
writting:  3145

writting:  1004462477  to:  10676
writting:  3147735591  to:  10680
writting:  3154423520  to:  10684
writting:  999469538  to:  10688
writting:  998790952  to:  10692
writting:  3152483880  to:  10696
writting:  985958155  to:  10700
writting:  3142977971  to:  10704
writting:  1000751144  to:  10708
writting:  3151916284  to:  10712
writting:  989185645  to:  10716
writting:  3139724097  to:  10720
writting:  3136883368  to:  10724
writting:  995354115  to:  10728
writting:  3132087335  to:  10732
writting:  3149911715  to:  10736
writting:  3157274686  to:  10740
writting:  3146500724  to:  10744
writting:  3146661280  to:  10748
writting:  3154637318  to:  10752
writting:  3152695916  to:  10756
writting:  3144572033  to:  10760
writting:  997783410  to:  10764
writting:  3132580169  to:  10768
writting:  985694841  to:  10772
writting:  992436190  to:  10776
writting:  3157565028  to:  10780
writting:  3153693678  to:  10784
writting:  1007581404  to:  10788
writting:  3153163821 

...

writting:  3156678501  to:  55256
writting:  3155275672  to:  55260
writting:  987739808  to:  55264
writting:  3130306471  to:  55268
writting:  3128578984  to:  55272
writting:  3156241852  to:  55276
writting:  3146350644  to:  55280
writting:  1006667563  to:  55284
writting:  3147321961  to:  55288
writting:  3153393181  to:  55292
writting:  992730955  to:  55296
writting:  1004765983  to:  55300
writting:  3150086211  to:  55304
writting:  1005448717  to:  55308
writting:  3137643985  to:  55312
writting:  3119499638  to:  55316
writting:  3154037771  to:  55320
writting:  3156598309  to:  55324
writting:  995786651  to:  55328
writting:  995269812  to:  55332
writting:  993638330  to:  55336
writting:  3148429896  to:  55340
writting:  3074403484  to:  55344
writting:  3155395379  to:  55348
writting:  3148046507  to:  55352
writting:  3144365438  to:  55356
writting:  3154576464  to:  55360
writting:  3149252459  to:  55364
writting:  3163763021  to:  55368
writting:  99981097

writting:  1023361921  to:  58620
writting:  1029784377  to:  58624
writting:  1026619561  to:  58628
writting:  3123493908  to:  58632
writting:  3151356126  to:  58636
writting:  1007399020  to:  58640
writting:  1003160667  to:  58644
writting:  3141369547  to:  58648
writting:  3136891948  to:  58652
writting:  3162505866  to:  58656
writting:  3158163636  to:  58660
writting:  3167763001  to:  58664
writting:  1008495467  to:  58668
writting:  1021595620  to:  58672
writting:  1020036650  to:  58676
writting:  1009347193  to:  58680
writting:  1010197779  to:  58684
writting:  3159945158  to:  58688
writting:  989946985  to:  58692
writting:  1004148191  to:  58696
writting:  998700599  to:  58700
writting:  3136006828  to:  58704
writting:  994709378  to:  58708
writting:  1008566854  to:  58712
writting:  1012185115  to:  58716
writting:  3158377370  to:  58720
writting:  1002673983  to:  58724
writting:  1025579338  to:  58728
writting:  1021203848  to:  58732
writting:  101091

writting:  968598757  to:  61980
writting:  3165335773  to:  61984
writting:  3166359455  to:  61988
writting:  3166389138  to:  61992
writting:  3148497616  to:  61996
writting:  988329027  to:  62000
writting:  1005362967  to:  62004
writting:  987205678  to:  62008
writting:  995351446  to:  62012
writting:  3100052643  to:  62016
writting:  3149102895  to:  62020
writting:  1014018616  to:  62024
writting:  3132026556  to:  62028
writting:  3149739506  to:  62032
writting:  3160450944  to:  62036
writting:  3152461305  to:  62040
writting:  3155813133  to:  62044
writting:  1006139770  to:  62048
writting:  3153797657  to:  62052
writting:  3135043425  to:  62056
writting:  1005132778  to:  62060
writting:  998333194  to:  62064
writting:  3154095881  to:  62068
writting:  1000039654  to:  62072
writting:  1001717447  to:  62076
writting:  3138647056  to:  62080
writting:  3133089376  to:  62084
writting:  3152646151  to:  62088
writting:  3144634444  to:  62092
writting:  31584054

writting:  943226579  to:  65336
writting:  3155427339  to:  65340
writting:  1004952041  to:  65344
writting:  3132433245  to:  65348
writting:  3114867224  to:  65352
writting:  1001770691  to:  65356
writting:  980978908  to:  65360
writting:  1007666263  to:  65364
writting:  3150693066  to:  65368
writting:  996956392  to:  65372
writting:  996525466  to:  65376
writting:  3157488780  to:  65380
writting:  3142105651  to:  65384
writting:  1014275709  to:  65388
writting:  1012047107  to:  65392
writting:  1013211508  to:  65396
writting:  3147272478  to:  65400
writting:  1007891297  to:  65404
writting:  989019696  to:  65408
writting:  999604168  to:  65412
writting:  1002708755  to:  65416
writting:  1002894305  to:  65420
writting:  984172308  to:  65424
writting:  990860660  to:  65428
writting:  3123564937  to:  65432
writting:  3136073985  to:  65436
writting:  1007715765  to:  65440
writting:  1021863650  to:  65444
writting:  1021973971  to:  65448
writting:  1023005788 

IndexError: index 16384 is out of bounds for axis 0 with size 16384

In [ ]:
%matplotlib inline

imgplot = plt.imshow(image)
plt.show()